In [1]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats
import sqlite3
import time

#Create db
conn = sqlite3.connect('financial_data.db')

# Create a cursor
cursor = conn.cursor()

value_invest_query = '''
CREATE TABLE value_invest (
        Ticker TEXT NON NULL,
        Price REAL, 
        Trailing_PE_ratio REAL, 
        Forward_PE_ratio REAL, 
        Price_To_Book REAL,
        Price_To_Sale REAL, 
        EV_EBITDA REAL, 
        EV_GP REAL, 
        Shares_to_buy REAL, 
        Trailing_PE_ratio_percentile REAL, 
        Forward_PE_ratio_percentile REAL,
        Price_To_Book_percentile REAL, 
        Price_To_Sale_percentile REAL, 
        EV_EBITDA_percentile REAL, 
        EV_GP_percentile REAL, 
        RV_Score REAL
);
'''

cursor.execute(value_invest_query)

conn.commit()


In [2]:
#Create first table with all stock info from yfinance
"""
Working capital has been removed because API calls are too slow. 
Also, working capital is somewhat linked to EBITDA
"""

tick = pd.read_csv("all_stocks")
value_invest_cols = ['Ticker','Price','Trailing PE ratio','Forward PE ratio','Price To Book',\
                                'Price To Sale','EV/EBITDA','EV/GP','Shares to buy']
value_invest = pd.DataFrame(columns = value_invest_cols)
count = 0 
for i in range(len(tick)): 
    start = time.time()    
    print("Producing data for: ", tick["0"][i])
    try:
        ticker = tick["0"][i]
        stock_info = yf.Ticker(ticker)
        d = yf.Ticker(ticker).history(period = "1mo")
        # Get the income statement for the stock

        info = stock_info.info
        fpe = info["forwardPE"]
        tpe = info["trailingPE"]
        fin = stock_info.info
        #liab = stock_info.balance_sheet
        price_to_book = fin['priceToBook']
        price_to_sale = fin['priceToSalesTrailing12Months']
        ev_ebidta = fin['enterpriseToEbitda']
        enterprise_value = fin['enterpriseValue']
        gp = fin['grossProfits']
        ev_gp = enterprise_value/gp
        #current_assets = liab[liab.index=='Current Assets'].iloc[0,0]
        #current_liab = liab[liab.index=='Current Liabilities'].iloc[0,0]
        #working_capital = current_assets/current_liab 

    except TypeError: 
        print("Nonetype found for: " + ticker)
        stop = time.time()
        duration = stop-start
        print(duration)
        continue
    except IndexError: 
        print("Couldn't find: ", ticker)
        stop = time.time()
        duration = stop-start
        print(duration)
        continue
    except KeyError:
        print("Couldnt find key for: " + ticker)
        stop = time.time()
        duration = stop-start
        print(duration)
        continue
    
    d.reset_index(inplace = True)
    d['Ticker'] = ticker
    d["Price"] = d['Close']
    d['Trailing PE ratio'] = tpe
    d['Forward PE ratio'] = fpe
    d['Price To Book'] = price_to_book
    d['Price To Sale'] = price_to_sale
    d['EV/EBITDA'] = ev_ebidta
    d['EV/GP'] = ev_gp
    d = d.iloc[-1]

    #Append to df
    value_invest = value_invest.append(
        pd.Series(d,
        index=value_invest_cols), 
        ignore_index = True
    )

    stop = time.time()
    duration = stop-start
    print(duration)
#Get percentiles for each stock and a specific category
cols = ['Trailing PE ratio', 'Forward PE ratio', 'Price To Book', 'Price To Sale', 'EV/EBITDA', 'EV/GP']
for c in cols: 
    value_invest[c + ' percentile'] = np.NaN
    for index, row in value_invest.iterrows(): 
        percentile_change = stats.percentileofscore(value_invest[c], value_invest[c].loc[index])
        value_invest[c + ' percentile'][index] = percentile_change 

value_invest['RV Score'] = 0

for index, row in value_invest.iterrows(): 
    value_invest['RV Score'][index] = (value_invest['Trailing PE ratio percentile'][index]\
        +value_invest['Forward PE ratio percentile'][index]\
        +value_invest['Price To Book percentile'][index]\
        +value_invest['Price To Sale percentile'][index]\
        +value_invest['EV/EBITDA percentile'][index]\
        +value_invest['EV/GP percentile'][index])/6

#PUSH TO DATABASE
value_invest.to_sql('value_invest', conn, if_exists='append', index=False)
# Close the connection
conn.close()

Producing data for:  AOS


<ipython-input-2-1956285887a8>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  value_invest = value_invest.append(


3.113816738128662
Producing data for:  ATEN


<ipython-input-2-1956285887a8>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  value_invest = value_invest.append(


1.9818329811096191
Producing data for:  AAC
Nonetype found for: AAC
1.6363089084625244
Producing data for:  AIR


...

2.0991320610046387
Producing data for:  AKR


<ipython-input-2-1956285887a8>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  value_invest = value_invest.append(


1.9729483127593994
Producing data for:  ACN


<ipython-input-2-1956285887a8>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  value_invest = value_invest.append(


2.479482412338257
Producing data for:  ACCO


In [ ]:
value_invest.columns

Index(['Ticker', 'Price', 'Trailing PE ratio', 'Forward PE ratio',
       'Price To Book', 'Price To Sale', 'EV/EBITDA', 'EV/GP', 'Shares to buy',
       'Trailing PE ratio percentile', 'Forward PE ratio percentile',
       'Price To Book percentile', 'Price To Sale percentile',
       'EV/EBITDA percentile', 'EV/GP percentile', 'RV Score'],
      dtype='object')